# Diabetes Prediction using KNN Classifier
В этом ноутбуке мы применяем метод классификации **KNN** для предсказания наличия диабета. Мы будем использовать набор данных, в котором содержится информация о пациентах, включая их медицинские данные, для того чтобы классифицировать, есть ли у них диабет (Outcome 1) или нет (Outcome 0).

## Шаги проекта:
1. Загрузка и первичная обработка данных.
2. Разделение данных на обучающую, валидационную и тестовую выборки.
3. Реализация KNN с нуля.
4. Оптимизация параметра `k` с помощью ручного подбора.
5. Использование `GridSearchCV` для автоматического выбора лучшего значения `k`.
6. Оценка модели с использованием метрик: Precision, Recall, F1-score и Accuracy.

In [16]:
# Загрузка необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

## Загрузка и первичная обработка данных
Здесь мы загружаем данные из файла `diabetes.csv` и сразу же просматриваем несколько первых строк для ознакомления с набором данных.

In [17]:
# Загрузка данных
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('diabetes.csv')

# Просмотр первых строк данных
data.head()

Saving diabetes.csv to diabetes (1).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Здесь мы считаем, как часто встречаются классы в нашем наборе данных (больной диабетом или нет).

In [18]:
# Подсчет доли классов в данных
class_counts = data['Outcome'].value_counts()
class_fractions = class_counts / len(data)
print(f"Class 0: {class_fractions[0]:.3f}\nClass 1: {class_fractions[1]:.3f}\n")

Class 0: 0.651
Class 1: 0.349



##Нормализация данных

In [19]:
# Функция для нормализации данных (z-score)
def z_score_norm(X):
    X_mean = X.mean()
    X_std = X.std()
    X_norm = (X - X_mean) / X_std
    return X_norm

# Нормализация данных без столбца 'Outcome' (потому что это целевая переменная)
X_norm = z_score_norm(data.drop('Outcome', axis=1))
X = X_norm
y = data['Outcome']

##Разделение данных на выборки
Разделяем данные на три части: обучающую выборку, валидационную и тестовую.

In [20]:
# Разделение на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

##Реализация KNN с нуля
Это  собственная реализация метода k ближайших соседей (KNN). Рассчитываем расстояния между объектами и выбираем `k` ближайших соседей для предсказания класса.


In [21]:
# Реализация функции KNN с нуля
def knn(X, y, X_new, k=5):
    y_pred = []
    for x_new in X_new:
        # Рассчитываем расстояния между новым объектом и всеми объектами обучающей выборки
        distances = np.sqrt(np.sum((X - x_new) ** 2, axis=1))
        # Находим индексы k ближайших соседей
        nearest_indices = np.argsort(distances)[:k]
        # Получаем метки классов ближайших соседей
        nearest_labels = y[nearest_indices]
        # Выбираем наиболее часто встречающийся класс среди соседей
        unique_classes, counts = np.unique(nearest_labels, return_counts=True)
        predicted_class = unique_classes[np.argmax(counts)]
        # Добавляем прогноз в список прогнозов
        y_pred.append(predicted_class)
    return np.array(y_pred)

##Оценка производительности модели
Функция `fscore` вычисляет основные метрики классификации: **precision**, **recall**, **f1-score** и **accuracy**.

In [22]:
# Функция для расчета метрик: precision, recall, f1_score, accuracy
def fscore(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recal = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recal) / (precision + recal) if (precision + recal) > 0 else 0
    accuracy = sum(y_true == y_pred) / len(y_true)

    return precision, recal, f1, accuracy

##Подбор оптимального значения k
Здесь мы вручную подбираем значение `k` для модели, чтобы найти оптимальное, которое даст наибольший **F1-score**.

In [23]:
# Определим оптимальное значение k с помощью ручного подбора
k_values = [i for i in range(1, 10)]
best_f1_score = 0
best_k = 0
for k in k_values:
    Y_val_pred = knn(X_train.values, y_train.values, X_val.values, k=k)
    _, _, f1score, _ = fscore(y_val, Y_val_pred)
    print(f"For k={k}, F1-score: {f1score:.3f}")
    if f1score > best_f1_score:
        best_f1_score = f1score
        best_k = k

print(f"Best F1-score: {best_f1_score:.3f} achieved with k={best_k}")

For k=1, F1-score: 0.625
For k=2, F1-score: 0.380
For k=3, F1-score: 0.642
For k=4, F1-score: 0.533
For k=5, F1-score: 0.661
For k=6, F1-score: 0.577
For k=7, F1-score: 0.642
For k=8, F1-score: 0.600
For k=9, F1-score: 0.655
Best F1-score: 0.661 achieved with k=5


##Использование GridSearchCV для подбора k
Теперь мы используем **GridSearchCV**, чтобы автоматически подобрать оптимальное значение `k` для классификатора.

In [24]:
# Использование GridSearchCV для подбора лучшего значения k
paramgrid = {'n_neighbors': range(1, 21)}
clf = KNeighborsClassifier(metric='euclidean')
grid_search = GridSearchCV(clf, paramgrid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

bestk = grid_search.best_params_['n_neighbors']
print("Best k:", bestk)

Best k: 9


##Оценка финальной модели
Наконец, мы оценим нашу модель на тестовой выборке с использованием метрик precision, recall, f1-score и accuracy.

In [26]:
# Оценка модели на тестовой выборке
clf = KNeighborsClassifier(n_neighbors=bestk, metric='euclidean')
clf.fit(X_train, y_train)
Y_test_pred = clf.predict(X_test)

precision = precision_score(y_test, Y_test_pred)
recall = recall_score(y_test, Y_test_pred)
f1score = f1_score(y_test, Y_test_pred)
accuracy = accuracy_score(y_test, Y_test_pred)

print(f"precision: {precision:.3f}")
print(f"recall: {recall:.3f}")
print(f"f1score: {f1score:.3f}")
print(f"accuracy: {accuracy:.3f}")

precision: 0.658
recall: 0.500
f1score: 0.568
accuracy: 0.753


##Заключение
В результате этого проекта мы разработали модель классификации на основе метода KNN для предсказания наличия диабета у пациентов. Мы протестировали модель с использованием как ручного подбора параметров, так и автоматического подбора с помощью **GridSearchCV**.